In [1]:
from SpectrogramDataset import SpectrogramDataset, MfccDataset, create_mfcc_dataset, create_dataframes, create_dataset
import torch
import numpy as np
import torch.nn as nn

In [2]:
file_path_df, track_df, genre_df = create_dataframes(file_paths_path = 'data/all_data_paths.txt' , 
                                                     tracks_csv_path = 'data/fma_metadata/tracks.csv', 
                                                     genre_csv_path = 'data/fma_metadata/genres.csv')


In [30]:
# train_data, validation_data, test_data = create_dataset(file_path_df, track_df, genre_df, test_percentage = .10, validation_percentage = .10)

train_data, validation_data, test_data = create_mfcc_dataset(file_path_df, track_df, genre_df, test_percentage = .10, validation_percentage = .10)



Failed to load  data/fma_small/108/108925.mp3
Failed to load  data/fma_small/099/099134.mp3
Training dataset created.
Validation dataset created.
Failed to load  data/fma_small/133/133297.mp3
Test dataset created.


In [38]:
class NN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels= 1, kernel_size = 3, stride = 1)
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3, stride = 1)
#         self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1)
        self.ReLU = nn.ReLU()
#         self.fc1 = nn.Linear(9280, 928)
#         self.fc2 = nn.Linear(928, 256)
        self.fc3 = nn.Linear(1869, 16)
        self.softmax = nn.Softmax(dim = 1)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = x[:,None, :, :]
        x = self.ReLU(self.conv1(x))
        x = self.maxpool(x)
        x = self.ReLU(self.conv2(x))
        x = self.maxpool(x)
#         x = self.ReLU(self.conv3(x))
#         x = self.maxpool(x)
        x = self.flatten(x)  
#         print(x.shape)
#         x = self.ReLU(self.fc1(x))
#         x = self.ReLU(self.fc2(x))
        x = self.softmax(self.fc3(x))
        
        return x

def evaluate(model, validation_loader, criterion):
    
    model.eval()
    
    val_loss = 0
    val_acc = 0
    
    for inputs, labels in validation_loader:
        outputs = model(inputs)
        val_loss += criterion(outputs, labels)
        _, true_labels = torch.max(labels, dim = 1)
        
        _, pred_labels = torch.max(outputs, dim = 1)
                
        val_acc += torch.tensor(torch.sum(pred_labels == true_labels).item()/len(pred_labels))
    
    val_loss = val_loss/len(validation_loader)
    val_acc = val_acc/len(validation_loader)
        
    return val_loss.detach().item(), val_acc.item()
        
        

def train_NN(model, train_loader, validation_loader, learning_rate:float, epochs:int=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        loss = 0
        for inputs, labels in train_loader:

            optimizer.zero_grad()
            
            outputs = model(inputs)
            train_loss = criterion(outputs, labels)

            train_loss.backward()

            optimizer.step()

            loss += train_loss.item()

        loss = loss/len(train_loader)
        
        val_loss, val_acc = evaluate(model, validation_loader, criterion)

        print("epoch: {}/{}, train loss = {:.6f}, val loss = {:.6f}, val acc = {:.6f}".format(epoch+1, epochs, loss, val_loss, val_acc))

    return None



In [41]:
# print(train_data.shape)
# train_data_tensor = torch.tensor(train_data)
# print(train_data_tensor.shape)
model = NN()
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size = 10, shuffle = False)
train_NN(model, train_loader, validation_loader, learning_rate = 1e-3, epochs = 50)

epoch: 1/50, train loss = 2.717969, val loss = 2.718865, val acc = 0.112500
epoch: 2/50, train loss = 2.713457, val loss = 2.713205, val acc = 0.111111
epoch: 3/50, train loss = 2.712883, val loss = 2.712090, val acc = 0.112500
epoch: 4/50, train loss = 2.711627, val loss = 2.716932, val acc = 0.112500
epoch: 5/50, train loss = 2.712237, val loss = 2.711705, val acc = 0.112500
epoch: 6/50, train loss = 2.711093, val loss = 2.727983, val acc = 0.112500
epoch: 7/50, train loss = 2.710013, val loss = 2.718814, val acc = 0.112500
epoch: 8/50, train loss = 2.710401, val loss = 2.710261, val acc = 0.112500
epoch: 9/50, train loss = 2.709114, val loss = 2.721027, val acc = 0.112500
epoch: 10/50, train loss = 2.708561, val loss = 2.709153, val acc = 0.127778
epoch: 11/50, train loss = 2.707309, val loss = 2.713187, val acc = 0.115278
epoch: 12/50, train loss = 2.707645, val loss = 2.710267, val acc = 0.116667
epoch: 13/50, train loss = 2.705996, val loss = 2.706559, val acc = 0.131944
epoch: 1

In [42]:
criterion = torch.nn.CrossEntropyLoss()
test_loader = torch.utils.data.DataLoader(validation_data, batch_size = 1, shuffle = False)
loss, acc = evaluate(model, test_loader, criterion)
print(loss, acc)

2.6441901433799 0.22777777910232544
